# Import Required Libraries

In [ ]:
import os
import time
import selenium
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Create Function for Scraping Account Summary and Portfolio Details

In [ ]:
def colfinancial_account_info(userkey1,userkey2,password):
    op = webdriver.ChromeOptions()
    op.add_argument('headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=op)
    url = "https://www.colfinancial.com/ape/Final2/home/HOME_NL_MAIN.asp?p=0"
    driver.get(url)
    time.sleep(3)
    username1 = driver.find_element_by_name("txtUser1")
    username2 = driver.find_element_by_name("txtUser2")
    pw = driver.find_element_by_name("txtPassword")
    username1.send_keys(userkey1)
    username2.send_keys(userkey2)
    pw.send_keys(password)
    
    driver.find_element_by_xpath("//*[@id='login']/div/input[10]").click()
    time.sleep(5)
    try:
        WebDriverWait(driver, 3).until(EC.alert_is_present(),
                                       'Timed out waiting for PA creation ' +
                                       'confirmation popup to appear.')

        alert = driver.switch_to.alert
        alert.accept()
        print("password change alert encountered...")
        print("password change alert accepted")
    except TimeoutException:
        print("no password change alert")

    driver.switch_to.frame("headern")
    driver.find_element_by_xpath("//*[@id='CT']/a").click()
    time.sleep(5)
    driver.find_element_by_xpath("//*[@id='L1_3_4']/a").click()
    time.sleep(5)
    driver.switch_to.parent_frame()
    driver.switch_to.frame("main")
    cash_balance = driver.find_element_by_xpath("//table[1]/tbody/tr[3]/td[2]")
    cash_balance = float(cash_balance.text.replace(',', ''))
    total_account_value = driver.find_element_by_xpath("//table/tbody/tr/td/p[1]/font[2]/strong")
    total_account_value = float(total_account_value.text.replace(',', ''))
    portfolio_table = driver.find_element_by_xpath("//html/body/div[2]/table/tbody/tr/td/table[2]/tbody/tr[3]/td")
    table = pd.read_html(portfolio_table.get_attribute("innerHTML"))[0]
    table = table.iloc[1:,1:-1]
    table = table[~table[1].isin(['TOTAL MUTUAL FUNDS','TOTAL MUTUAL FUNDS GAIN/LOSS','NaN','Fund Code','MUTUAL FUNDS'])]
    table = table.dropna()
    equity_portfolio = table[~table[1].isin(['TOTAL EQUITIES','TOTAL EQUITIES GAIN/LOSS','TOTAL PORTFOLIO TRADE VALUE:','DAY CHANGE:','PORTFOLIO GAIN/LOSS:'])]
    summary_table = table.drop(equity_portfolio.index, axis=0)
    summary_table = summary_table.iloc[:,6:8]
    summary_table.columns = ['Item', 'Value']
    equity_portfolio.columns = equity_portfolio.iloc[0]
    equity_portfolio = equity_portfolio[1:]
    summary_table.index = summary_table['Item'] 
    summary_table= summary_table.iloc[:,1:]
    summary_table['Value'] = summary_table['Value'].str.replace("%","", regex=True).astype(float)
    total_equity_portfolio_value = summary_table.loc['TOTAL EQUITIES']['Value']
    driver.quit()
    return equity_portfolio, cash_balance, total_equity_portfolio_value, total_account_value
    
    

# Retrieve Our Account Summary

In [ ]:
userkey1 = 'XXXX'    # Input the first 4 digit of your 8-digit User ID
userkey2 = 'XXXX'    # Input the first 4 digit of your 8-digit User ID
password = "XXXXXXXXXX"    # Input your password
portfolio_df, cash, equity, total = colfinancial_account_info(userkey1,userkey2,password)

# Print Account Summary

In [ ]:
print("Cash Balance: {}".format(cash))
print("Equity Portfolio Value: {}".format(equity))
print("Total Account Balance: {}".format(total))

# Print Out Equity Portfolio Details

In [ ]:
portfolio_df